# Week 09 — Time Series Trend & Seasonality (트렌드/계절성)

이 노트북은 **실전 통계 → 해석 → 비즈니스 액션**을 훈련하기 위한 템플릿입니다.
각 섹션의 ✅ 체크포인트를 채우면서 진행하세요.


## 1) Business Questions
- 최근 성과가 추세 변화인지, 계절성/노이즈인지 구분할 수 있는가?
- 이상치 제거 후에도 추세가 유지되는가?

## 2) Why this matters (Theory)
- 시계열은 추세(trend) + 계절(seasonality) + 잔차(residual)로 해석
- 이동평균과 로그 변환으로 해석 안정성 강화
- 이상치(프로모션) 제거 후에도 같은 결론인지 확인

## 3) Setup & Data Load

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

DATA_PATH = "data/sales_marketing_example_2025.csv"
df = pd.read_csv(DATA_PATH, parse_dates=["date"])

# Quick sanity checks
display(df.head())
print("rows, cols:", df.shape)
print("missing rate (top):")
display(df.isna().mean().sort_values(ascending=False).head(10))

## 4) Helper Functions

In [ ]:
def one_sentence_insight(prompt: str):
    """Write your one-sentence business insight below."""
    print("\n🧾 One-sentence insight (fill this):")
    print(" -", prompt)

def bootstrap_mean_diff(x, y, iters=5000, seed=0):
    """Bootstrap CI for difference in means (x - y)."""
    r = np.random.default_rng(seed)
    x = np.asarray(pd.Series(x).fillna(0))
    y = np.asarray(pd.Series(y).fillna(0))
    diffs = []
    for _ in range(iters):
        xs = r.choice(x, size=len(x), replace=True)
        ys = r.choice(y, size=len(y), replace=True)
        diffs.append(xs.mean() - ys.mean())
    diffs = np.array(diffs)
    return diffs.mean(), np.quantile(diffs, [0.025, 0.975])

def rate_ci_wilson(successes, n, alpha=0.05):
    """Wilson score interval for a proportion."""
    if n == 0:
        return (np.nan, np.nan)
    z = stats.norm.ppf(1 - alpha/2)
    phat = successes / n
    denom = 1 + z**2/n
    center = (phat + z**2/(2*n)) / denom
    half = z * np.sqrt((phat*(1-phat) + z**2/(4*n)) / n) / denom
    return center - half, center + half

---
## 5) Analysis

### 5.1 Daily revenue trend with smoothing
✅ 이동평균으로 큰 그림을 봅니다.

In [ ]:
daily = (df.assign(revenue=df['revenue'].fillna(0))
           .groupby('date')['revenue'].sum()
           .sort_index())
ma7 = daily.rolling(7).mean()
ma28 = daily.rolling(28).mean()

plt.figure()
plt.plot(daily.index, daily.values, linewidth=0.6)
plt.plot(ma7.index, ma7.values, linewidth=1.2)
plt.plot(ma28.index, ma28.values, linewidth=1.6)
plt.title("Daily revenue with 7/28-day moving averages")
plt.show()

In [ ]:
one_sentence_insight('7일/28일 이동평균 기준으로 성과가 좋아졌는지 한 문장')

### 5.2 Outlier-robust trend (clip at 99th pct per day)
✅ 이상치가 트렌드를 왜곡하지 않게 클리핑합니다.

In [ ]:
tmp = df.copy()
tmp['revenue_c'] = tmp['revenue'].fillna(0).clip(upper=tmp['revenue'].fillna(0).quantile(0.99))
daily_c = tmp.groupby('date')['revenue_c'].sum().sort_index()
daily_c.rolling(28).mean().plot(title="Clipped revenue (99th pct) — 28-day MA")
plt.show()

In [ ]:
one_sentence_insight('이상치 제거 후에도 같은 결론인지 한 문장')

---
## 6) Interpretation Checklist
- 결과가 '유의'하면, **효과 크기/CI**가 실무적으로 의미 있는가?
- 결과가 '비유의'하면, **표본 부족/측정오류/세그먼트 혼합** 가능성은?
- 액션을 한다면, **누구에게/어디에/얼마나** 적용할 것인가?



## Executive Summary (메일/슬라이드에 그대로 붙여넣기)

- **핵심 질문:**  
- **결론(한 문장):**  
- **근거 지표:** (효과크기 / p-value / CI 등)  
- **비즈니스 임팩트:** (예: 월 매출 +X, 마진 -Y 위험)  
- **권장 액션:** (실험/타겟 변경/채널 믹스 조정)  
- **추가 분석 필요:**  
